# What is this notebook about?
This notebookstands for testing and demostrating pipeline of BPMN ai agent creation

## simple check if everything working fine
If it is there will be text from the model

In [1]:
from openai import OpenAI
from settings import get_settings # set up scipt before testing

settings = get_settings()


AI_API_KEY = settings.OPENROUTER_API_KEY
MODEL_NAME = settings.OPENROUTER_MODEL_NAME

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=AI_API_KEY,
)

completion = client.chat.completions.create(
    extra_headers={
        "HTTP-Referer": "<YOUR_SITE_URL>",  # Optional. Site URL for rankings on openrouter.ai.
        "X-Title": "<YOUR_SITE_NAME>",  # Optional. Site title for rankings on openrouter.ai.
    },
    extra_body={},
    model=MODEL_NAME,
    messages=[{"role": "user", "content": "What is the meaning of life?"}],
    stream=True,
)
full_response = ""
for chunk in completion:
    if chunk.choices[0].delta.content:
        full_response += chunk.choices[0].delta.content
        print(chunk.choices[0].delta.content, end="", flush=True)


The question *"What is the meaning of life?"* is one of the most profound and enduring philosophical inquiries. There is no single, universally accepted answer, but different perspectives offer insights:

### **1. Philosophical Perspectives**
- **Existentialism (Sartre, Camus, Nietzsche):** Life has no inherent meaning—it’s up to each individual to create their own purpose through choices, actions, and passions.
- **Absurdism (Camus):** Life may lack intrinsic meaning, but we must embrace the absurd and find joy in the struggle itself.
- **Stoicism:** Meaning comes from living virtuously, accepting what we cannot control, and focusing on reason and self-improvement.
- **Utilitarianism:** The meaning of life is to maximize happiness and reduce suffering for the greatest number of beings.

### **2. Religious & Spiritual Views**
- **Christianity/Judaism/Islam:** Life’s purpose is to serve, love, and connect with God, following divine will.
- **Buddhism/Hinduism:** Meaning comes from enlig

# Graph pipeline
Lets start the experiments with pipline

In [2]:
import json
from settings import get_settings
from openai import OpenAI
from src.ai_generation.llm_client import LLMClient

settings = get_settings()


AI_API_KEY = settings.OPENROUTER_API_KEY
MODEL_NAME = settings.OPENROUTER_MODEL_NAME

raw_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=AI_API_KEY,
)

client = LLMClient(raw_client,MODEL_NAME, "Ты полезный ассистент")

full_response = ""

## 1. Schema planning

firstly generate needed schema for the task

In [3]:
with open(r'data\bpmn_schemas\plan.json', 'r', encoding='utf-8') as file:
    json_schema = json.load(file)

base_promt = "Write a BPMN schema for the task: Logistical company is planning to create a new product"
response = client.generate_response_json_based(prompt=base_promt, json_schema=json_schema)
print(json.dumps(json.loads(response), indent=4))

full_response += response

{
    "intent": "bpmn_diagram",
    "description": "BPMN diagram for a logistical company planning to create a new product",
    "options": {
        "collaboration": true,
        "autoLayout": true,
        "includeDI": true
    }
}


## 2. Indetify nodes (processes) for task

In [4]:
with open(r'data\bpmn_schemas\process.json', 'r', encoding='utf-8') as file:
    json_schema = json.load(file)

response_processes = client.generate_response_json_based(prompt=full_response, json_schema=json_schema)
print(json.dumps(json.loads(response_processes), indent=4))

full_response += response_processes

{
    "id": "logistics_product_launch",
    "name": "New Product Launch Process",
    "nodes": [
        {
            "id": "start",
            "name": "Product Idea Initiation",
            "type": "startEvent"
        },
        {
            "id": "task1",
            "name": "Market Research",
            "type": "task"
        },
        {
            "id": "task2",
            "name": "Feasibility Study",
            "type": "userTask"
        },
        {
            "id": "gateway1",
            "name": "Feasibility Check",
            "type": "exclusiveGateway"
        },
        {
            "id": "task3",
            "name": "Product Design",
            "type": "task"
        },
        {
            "id": "task4",
            "name": "Prototype Development",
            "type": "userTask"
        },
        {
            "id": "task5",
            "name": "Supplier Selection",
            "type": "serviceTask"
        },
        {
            "id": "gateway2",
         

## 3. Indetify flows
Create flows for the task nodes

In [5]:
with open(r'data\bpmn_schemas\flow.json', 'r', encoding='utf-8') as file:
    json_schema = json.load(file)

response_flows = client.generate_response_json_based(prompt=full_response, json_schema=json_schema)
print(json.dumps(json.loads(response_flows), indent=4))

full_response += response_flows

{
    "flows": [
        {
            "id": "flow1",
            "sourceRef": "start",
            "targetRef": "task1"
        },
        {
            "id": "flow2",
            "sourceRef": "task1",
            "targetRef": "task2"
        },
        {
            "id": "flow3",
            "sourceRef": "task2",
            "targetRef": "gateway1"
        },
        {
            "id": "flow4",
            "sourceRef": "gateway1",
            "targetRef": "task3",
            "name": "Feasible"
        },
        {
            "id": "flow5",
            "sourceRef": "gateway1",
            "targetRef": "end",
            "name": "Not Feasible"
        },
        {
            "id": "flow6",
            "sourceRef": "task3",
            "targetRef": "task4"
        },
        {
            "id": "flow7",
            "sourceRef": "task4",
            "targetRef": "gateway2"
        },
        {
            "id": "flow8",
            "sourceRef": "gateway2",
            "targetRef": "

## 4. (OPTIONAL) Add collaboration
If there are some participants in the task, add collaboration

In [6]:
with open(r'data\bpmn_schemas\collaboration.json', 'r', encoding='utf-8') as file:
    json_schema = json.load(file)
    
response_collaboration = client.generate_response_json_based(prompt=full_response, json_schema=json_schema)
print(json.dumps(json.loads(response_collaboration), indent=4))

full_response += response_collaboration

{
    "id": "logistics_product_launch",
    "participants": [
        {
            "id": "market_research_team",
            "name": "Market Research Team",
            "processRef": "task1"
        },
        {
            "id": "feasibility_analyst",
            "name": "Feasibility Analyst",
            "processRef": "task2"
        },
        {
            "id": "product_design_team",
            "name": "Product Design Team",
            "processRef": "task3"
        },
        {
            "id": "prototype_developer",
            "name": "Prototype Developer",
            "processRef": "task4"
        },
        {
            "id": "supplier_manager",
            "name": "Supplier Manager",
            "processRef": "task5"
        },
        {
            "id": "production_team",
            "name": "Production Team",
            "processRef": "task6"
        },
        {
            "id": "marketing_team",
            "name": "Marketing Team",
            "processRef": "task7

## 5. (OPTIONAL) diagramm rendering / Logical DI
If there are some logical diagrams, add them to the task

In [7]:
with open(r'data\bpmn_schemas\diagramm_schema.json', 'r', encoding='utf-8') as file:
    json_schema = json.load(file)

response_di = client.generate_response_json_based(prompt=full_response, json_schema=json_schema)
print(json.dumps(json.loads(response_di), indent=4))

full_response += response_di

{
    "id": "logistics_product_launch",
    "participants": [
        {
            "id": "market_research_team",
            "name": "Market Research Team",
            "processRef": "task1"
        },
        {
            "id": "feasibility_analyst",
            "name": "Feasibility Analyst",
            "processRef": "task2"
        },
        {
            "id": "product_design_team",
            "name": "Product Design Team",
            "processRef": "task3"
        },
        {
            "id": "prototype_developer",
            "name": "Prototype Developer",
            "processRef": "task4"
        },
        {
            "id": "supplier_manager",
            "name": "Supplier Manager",
            "processRef": "task5"
        },
        {
            "id": "production_team",
            "name": "Production Team",
            "processRef": "task6"
        },
        {
            "id": "marketing_team",
            "name": "Marketing Team",
            "processRef": "task7

## 6. Layout

Can be migrated into script

In [8]:
with open(r'data\bpmn_schemas\layout.json', 'r', encoding='utf-8') as file:
    json_schema = json.load(file)

response_layout = client.generate_response_json_based(prompt=full_response, json_schema=json_schema)
print(json.dumps(json.loads(response_layout), indent=4))

full_response += response_layout

{
    "positions": [
        {
            "elementId": "start",
            "bounds": {
                "x": 50,
                "y": 50,
                "width": 36,
                "height": 36
            }
        },
        {
            "elementId": "task1",
            "bounds": {
                "x": 50,
                "y": 120,
                "width": 100,
                "height": 80
            }
        },
        {
            "elementId": "task2",
            "bounds": {
                "x": 50,
                "y": 230,
                "width": 100,
                "height": 80
            }
        },
        {
            "elementId": "gateway1",
            "bounds": {
                "x": 50,
                "y": 340,
                "width": 50,
                "height": 50
            }
        },
        {
            "elementId": "task3",
            "bounds": {
                "x": 50,
                "y": 420,
                "width": 100,
                "h

# 8. (Optional) Validation

Add sooner

# BPMN generating
for now - simply send the response in json to the AI -> get XML -> generate BPMN by bpmn-js based on XML

In [11]:
with open(r"data\XMLs\base_bpmn_diagram.xml", "r") as file:
    refer_xml = file.read()

xml = client.generate_response_text_based(prompt=f"""
                                          Rewrite the response in json format to the XML code:\n
                                          {full_response}
                                          Use this xml as a reffer of valid xml\n: 
                                          {refer_xml}
                                          """)
print("succesfull")

succesfull


In [ ]:
with open(r"data\XMLs\test_schemas\bpmn_test_schema.xml", "w") as file:
    file.write(xml)